In [1]:
#selenium 사용

from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome('../driver/chromedriver')
driver.get('https://topis.seoul.go.kr/')

#창 로드 완료를 기다리기 위한 time.sleep 3초
time.sleep(3)

#홈페이지 내 팝업 창을 끄기 위한 selenium.webdriver.common.keys 사용
element = driver.find_element_by_xpath('//*[@id="pop03-closer"]')
driver.execute_script("arguments[0].click();", element)

#버스 단추 선택
bus_list_raw = driver.find_element_by_xpath('/html/body/div[4]/a[3]')
bus_list_raw.click()

#1) 정류소 정보 excel 파일 받아오기
station_raw= driver.find_element_by_xpath('//*[@id="mapContainer"]/div[1]/div/div[2]/a[2]')
station_raw.click()

#1-1) 받아온 excel 파일에서 야간버스만 추출
import pandas as pd
bus=pd.read_excel('C:/Users/user/Downloads/stationlist.xlsx')
bus_test=pd.read_excel('C:/Users/user/Downloads/stationlist.xlsx',header=3)

#버스 노선만 추출
bus_cat=bus_df.iloc[:,0]

#중복 값 제거를 위한 Dataframe -> list -> set변환
bus_cat_val=bus_cat.values
bus_cat_list =bus_cat_val.tolist()
bus_cat_list_set = set(bus_cat_list)
bus_cat_list = list(bus_cat_list_set)
bus_cat_list #서울시 총 버스 노선

#야간 노선만 추출
night_bus=[]
for i in bus_cat_list:
    if 'N' in i:
        night_bus.append(i)
        
#night_bus와 비교 후 없는 노선 Nbus 변수에 따로 추가
night_bus
Nbus = ['N13', 'N15','N16','N26','N30','N32','N34','N37'
        ,'N51','N61','N62','N64','N72','N75']

#각 버스 검색후 정류장 정보 table 받아오기
temp=[]
for i in Nbus:
    #각 버스 검색창에 넣기
    search_bus = driver.find_element_by_id('routName')
    search_bus.clear()
    search_bus.send_keys(i)
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="searchForm"]/div/ul/li/button').click()
    time.sleep(1)
    element = driver.find_element_by_xpath('//*[@id="resultList"]/li[1]/div[1]/a[2]')
    driver.execute_script("arguments[0].click();", element)
    time.sleep(1)
    driver.window_handles

    #받아온 정보 노선별 정류장 stat에 추가 후 전체 정보를 temp에 추가
    stat=[]
    for j in range(1,4):
        driver.switch_to.window(driver.window_handles[-1])
        table = driver.find_element_by_xpath('//*[@id="pop_content"]/div[3]/div['+str(j)+']/table')
        tbody = table.find_element_by_tag_name('tbody')
        for tr in tbody.find_elements_by_tag_name("tr"):
            for td in tr.find_elements_by_tag_name("td"):                
                txt = td.get_attribute("innerText")
                stat.append(txt)
            for th in tr.find_elements_by_tag_name("th"):
                txt = th.get_attribute("innerText")
                stat.append(txt)

            print(txt)
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(2)    
    temp.append(stat)
#다른 창으로 이동 방지
driver.switch_to.window(driver.window_handles[0])

        
#index와 버스정류장 이름 각각 리스트에 저장
for i,Nb in zip(temp, Nbus):
    globals()[Nb] = []
    ind = []
    nm = []
    for j in range(len(i)):
        if j % 2 == 1:
            nm.append(i[j])            
        else:
            ind.append(i[j])
    globals()[Nb].append(ind)
    globals()[Nb].append(nm)
print(nm)

#버스정류장 정보 csv로 저장
for nb in Nbus:
    print()
    df = pd.DataFrame({'정류소이름' : globals()[nb][1]}, index=globals()[nb][0])
    df.to_csv('정류소_' + nb + '.csv')

#데이터 확인
pd.read_csv('정류소_N13.csv')


WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [2]:
#승객수 표시
import pandas as pd

#필요 데이터 열 확인 후 추출
bong = pd.read_csv('C:/Users/user/Documents/카카오톡 받은 파일/bongcheon.csv',usecols=[1,3])
hwa = pd.read_csv('C:/Users/user/Documents/카카오톡 받은 파일/hwagoak.csv',usecols=[1,3])
mok = pd.read_csv('C:/Users/user/Documents/카카오톡 받은 파일/mok.csv',usecols=[1,3])
sang = pd.read_csv('C:/Users/user/Documents/카카오톡 받은 파일/sangdo.csv',usecols=[1,3])
sin = pd.read_csv('C:/Users/user/Documents/카카오톡 받은 파일/sinlim.csv',usecols=[1,3])
yeok = pd.read_csv('C:/Users/user/Documents/카카오톡 받은 파일/yeoksam.csv',usecols=[1,3])
bus_sta = pd.read_csv('C:/Users/user/Documents/카카오톡 받은 파일/버스정류장 정보.csv')

#데이터프레임 합침
seoul = pd.concat([bong,hwa,mok,sang,sin,yeok])

#'정류소명'을 키로 합치기 위하여 열이름 변경
seoul.rename(columns={'역명':'정류소명'},inplace=True)

#merge를 이용하여 각 정류소별 좌표와 승객 수 확인(22시 이후)
bus_station = pd.merge(seoul,bus_sta,how='inner',on='정류소명')

#지도에 표시하기 위한 각 필요 열 list로 변환
X=list(bus_station['좌표X'])
Y=list(bus_station['좌표Y'])
station = list(bus_station['정류소명'])
pas = list(bus_station['승객수'])

#folium을 통한 승객수 표시(원의 크기), start 좌표
import folium
m = folium.Map(location=[37.55, 126.986],zoom_start=17)
for i, j, k, l in zip(X,Y,station,pas):
    folium.CircleMarker([j, i],popup=k,radius=l*0.001).add_to(m)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/user/Documents/카카오톡 받은 파일/bongcheon.csv'